In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from IPython import get_ipython
from IPython.display import display
from datasets import load_dataset
import pandas as pd

ds = pd.read_json("hf://datasets/Amod/mental_health_counseling_conversations/combined_dataset.json", lines=True)
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...
...,...,...
3507,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...
3508,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...
3509,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit..."
3510,I think adult life is making him depressed and...,How do you help yourself to believe you requir...


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
import torch

model_name = "meta-llama/Llama-3.2-1B-Instruct"
device = "cuda" #Metal Performance Shaders
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="right") #padding_side show where padding tokens should be added to sequences
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            torch_dtype=torch.bfloat16,
                                            device_map=device)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [4]:
generation_pipeline = pipeline(task="text-generation", #pipeline เชื่อมต่อโมเดลกับขั้นตอนอื่น ๆ
                               model=model, tokenizer=tokenizer)
generation_pipeline("Hello", max_new_tokens=25)

Device set to use cuda


[{'generated_text': "Hello, I am a bit of a tech enthusiast, and I am curious about the world of AI and machine learning. I've"}]

In [5]:
prompts = [
    "hello how are you?",
    "The nigger"
]

tokenized = tokenizer(prompts,padding=True, return_tensors="pt").to(device)

print(tokenized)
print(tokenized["input_ids"].shape)

{'input_ids': tensor([[128000,  15339,   1268,    527,    499,     30],
        [128000,    791,    308,   4601, 128009, 128009]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0]], device='cuda:0')}
torch.Size([2, 6])


In [6]:
tokenizer.batch_decode(tokenized["input_ids"])

['<|begin_of_text|>hello how are you?',
 '<|begin_of_text|>The nigger<|eot_id|><|eot_id|>']

In [7]:
tokenized["attention_mask"] #tell which have value

tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0]], device='cuda:0')

In [8]:
prompt_template = [
    {
      "role" : "system",
      "content" : "You are a smart AI assistant"
    },
    {
        "role" : "user",
        "content" : "Where does the sun rises?"
    },
    {
        "role" : "assistant",
        "content" : "Aye aye"
    }
]

tokenizer.pad_token = tokenizer.eos_token

tokenized = tokenizer.apply_chat_template( #converts prompt from chat message format to a single string sequence
    prompt_template,
    add_generation_prompt=True,
    tokenize=True, #if set True, will get number that easily insert to model
    padding=True,
    return_tensors="pt"
)

print(tokenized)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1591,   3297,    220,   2366,     20,    271,   2675,    527,
            264,   7941,  15592,  18328, 128009, 128006,    882, 128007,    271,
           9241,   1587,    279,   7160,  38268,     30, 128009, 128006,  78191,
         128007,    271,     32,   9188,    264,   9188, 128009, 128006,  78191,
         128007,    271]])


In [9]:
out = model.generate(tokenized.to(device), max_new_tokens=30)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [10]:
decoded = tokenizer.batch_decode(out)
print(decoded[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 28 May 2025

You are a smart AI assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Where does the sun rises?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Aye aye<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The sun rises in the eastern hemisphere of the Earth, specifically at the location where the Earth's axis is tilted towards the Sun. This is because the


In [11]:
import torch.nn as nn

predict next word

In [12]:
 sentence = "you are game"
 input_ids = tokenizer([sentence], return_tensors="pt")["input_ids"].to(device) # return_tensors use to change type to tensor from list

 out = model(input_ids = input_ids) # model(input) does one step of generation by outputting the probabilities of next word
 out

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 2.8438,  3.5781,  7.0312,  ..., -1.2422, -1.2422, -1.2422],
         [ 8.0625,  9.1875,  7.5312,  ..., -0.8984, -0.8984, -0.8984],
         [ 8.1250,  7.3438,  4.0938,  ..., -3.1562, -3.1562, -3.1562],
         [12.8125,  7.5938,  8.3125,  ..., -2.3750, -2.3750, -2.3750]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x7d459ad9bc50>, hidden_states=None, attentions=None)

In [13]:
# predict what is the next word
probability_dist = nn.Softmax(dim=-1)(out.logits[0, -1]) # softmax change to probability
print(out.logits)
print("_____________________________")
print(out.logits.shape) # models store millions of subwords, vocab has 128256 tokens
print("_____________________________")
print(input_ids)
print("_____________________________")
print(probability_dist)
print("_____________________________")
print(out.logits.argmax(), out.logits.argmax(axis=-1)) #argmax find maximum value in the list
print(tokenizer.decode(out.logits.argmax(axis=-1)[0, -1]))

tensor([[[ 2.8438,  3.5781,  7.0312,  ..., -1.2422, -1.2422, -1.2422],
         [ 8.0625,  9.1875,  7.5312,  ..., -0.8984, -0.8984, -0.8984],
         [ 8.1250,  7.3438,  4.0938,  ..., -3.1562, -3.1562, -3.1562],
         [12.8125,  7.5938,  8.3125,  ..., -2.3750, -2.3750, -2.3750]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)
_____________________________
torch.Size([1, 4, 128256])
_____________________________
tensor([[128000,   9514,    527,   1847]], device='cuda:0')
_____________________________
tensor([4.9744e-03, 2.6941e-05, 5.5313e-05,  ..., 1.2660e-09, 1.2660e-09,
        1.2660e-09], device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<SoftmaxBackward0>)
_____________________________
tensor(256776, device='cuda:0') tensor([[16309,   649,   264,   311]], device='cuda:0')
 to


loss function (training on sequence)

In [14]:
sentence = ["use teng service instead."]
tokenized = tokenizer(sentence, return_tensors="pt")
print(tokenized)
tokenized2 = tokenizer(sentence, return_tensors="pt")["input_ids"] #print only input_ids
print(tokenized2)
print(tokenizer.batch_decode(tokenized2))

{'input_ids': tensor([[128000,    817,  42249,   2532,   4619,     13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
tensor([[128000,    817,  42249,   2532,   4619,     13]])
['<|begin_of_text|>use teng service instead.']


In [15]:
input_ids = tokenized["input_ids"][:, :-1]
target_ids = tokenized["input_ids"][:, 1:]

print("Input Seq: ", input_ids)
print("Target Seq: ", tokenizer.batch_decode(input_ids))
print(target_ids)
print(tokenizer.batch_decode(target_ids))

Input Seq:  tensor([[128000,    817,  42249,   2532,   4619]])
Target Seq:  ['<|begin_of_text|>use teng service instead']
tensor([[  817, 42249,  2532,  4619,    13]])
['use teng service instead.']


In [16]:
ques = "what service should I use?"
answer = "Teng Service"

prompt = [
    {"role": "user", "content": "what service should I use?"},
    {"role": "assistant", "content": "Service:"}
]
answer = "Teng Service"

chat_template = tokenizer.apply_chat_template(
    prompt,
    continue_final_message=True,
    tokenize=False
)
print(chat_template)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 28 May 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

what service should I use?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Service:


In [17]:
full_response = chat_template + " " + answer + tokenizer.eos_token
print(full_response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 28 May 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

what service should I use?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Service: Teng Service<|eot_id|>


In [18]:
tokenized = tokenizer(full_response, return_tensors="pt", add_special_tokens=False)["input_ids"]
print(tokenized)
print(tokenizer.batch_decode(tokenized))

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1591,   3297,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  12840,   2532,   1288,    358,   1005,     30,
         128009, 128006,  78191, 128007,    271,   1898,     25,    350,    833,
           5475, 128009]])
['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 28 May 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nwhat service should I use?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nService: Teng Service<|eot_id|>']


In [19]:
input_ids = tokenized[:, :-1]
target_ids = tokenized[:, 1:]

print("Input Seq: ", input_ids)
print("Target Seq: ", tokenizer.batch_decode(input_ids))
print(target_ids)
print(tokenizer.batch_decode(target_ids))

Input Seq:  tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1591,   3297,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  12840,   2532,   1288,    358,   1005,     30,
         128009, 128006,  78191, 128007,    271,   1898,     25,    350,    833,
           5475]])
Target Seq:  ['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 28 May 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nwhat service should I use?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nService: Teng Service']
tensor([[128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,     25,
           6790,    220,   2366,     18,    198,  15724,   2696,     25,    220,
           1591,   3297,    220,   2366,     20,    271, 128009, 128006,    882,
         12800

In [20]:
# demonstrate of using tokenizer
print(tokenizer.convert_ids_to_tokens(32346))

Too


In [21]:
labels_tokenized = tokenizer([" " + answer], add_special_tokens=False, return_tensors="pt")["input_ids"] # answer in prompt
print(labels_tokenized)
print(tokenizer.batch_decode(labels_tokenized))

tensor([[ 350,  833, 5475]])
[' Teng Service']


In [22]:
input_ids.shape

torch.Size([1, 46])

In [23]:
labels_tokenized2 = tokenizer([" " + answer + tokenizer.eos_token], add_special_tokens=False, return_tensors="pt", padding="max_length", max_length=target_ids.shape[1])["input_ids"]
print(labels_tokenized2)

tensor([[   350,    833,   5475, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009]])


In [24]:
labels_tokenized_fixed = torch.where(labels_tokenized2 != tokenizer.pad_token_id, labels_tokenized2, -100)
labels_tokenized_fixed

tensor([[ 350,  833, 5475, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [25]:
import torch # Assuming torch is used for tensor operations

def generate_input_output_pair(prompt, target_responses):
    chat_templates = tokenizer.apply_chat_template(prompt, continue_final_message=True, tokenize=False)
    full_response_text = [
        chat_template + " " + target_response + tokenizer.eos_token
        for chat_template, target_response in zip(chat_templates, target_responses)
    ]

    input_ids_tokenized = tokenizer(full_response_text, return_tensors="pt", add_special_tokens=False)["input_ids"]

    labels_tokenized = tokenizer([
        "" + response + tokenizer.eos_token for response in target_responses
    ], add_special_tokens=False, return_tensors="pt", padding="max_length", max_length=input_ids_tokenized.shape[1])["input_ids"]

    labels_tokenized_fixed = torch.where(labels_tokenized == tokenizer.pad_token_id, labels_tokenized, -100)
    labels_tokenized_fixed[:, -1] = tokenizer.pad_token_id

    input_ids_tokenized_left_shifted = input_ids_tokenized[:, :-1]
    labels_tokenized_right_shifted = labels_tokenized_fixed[:, 1:]

    attention_mask = input_ids_tokenized_left_shifted != tokenizer.pad_token_id

    return input_ids_tokenized_left_shifted, labels_tokenized_right_shifted, attention_mask

In [26]:
import torch # Assuming torch is used for tensor operations

def generate_input_output_pair_test(prompt, target_responses):
    chat_templates = tokenizer.apply_chat_template(prompt, continue_final_message=True, tokenize=False)
    full_response_text = [
        chat_template + " " + target_response + tokenizer.eos_token
        for chat_template, target_response in zip(chat_templates, target_responses)
    ]

    input_ids_tokenized = tokenizer(full_response_text, return_tensors="pt", add_special_tokens=False)["input_ids"]

    labels_tokenized = tokenizer([
        "" + response + tokenizer.eos_token for response in target_responses
    ], add_special_tokens=False, return_tensors="pt", padding="max_length", max_length=input_ids_tokenized.shape[1])["input_ids"]

    # The original code was marking pad tokens in labels as -100, but also the last token
    # Let's revisit the logic for labels to ensure correct loss calculation.
    # For causal language modeling, the labels are typically the input IDs shifted by one position.
    # We only want to calculate the loss for the tokens corresponding to the target response.
    # Tokens before the target response and padding tokens should be masked with -100.

    # Create labels by shifting input_ids_tokenized
    labels = input_ids_tokenized.clone()
    # Shift labels to the left (the target for index i is the token at index i+1)
    labels[:, :-1] = input_ids_tokenized[:, 1:]
    # The last label should be -100 or the pad token id depending on desired behavior
    # For now, let's set it to -100 to match the original masking intent for non-response tokens
    labels[:, -1] = -100 # Assuming the last token should not contribute to the loss calculation for the label sequence

    # Create a mask to identify the tokens corresponding to the target response in the labels
    # This requires knowing where the target response starts in the full response text.
    # A more robust approach might be to tokenized the target response separately
    # and use its tokenized length to mask the labels tensor.

    # Let's try masking based on the tokenized target response length
    target_response_tokenized = tokenizer([response + tokenizer.eos_token for response in target_responses], add_special_tokens=False, return_tensors="pt")["input_ids"]
    target_response_length = target_response_tokenized.shape[1]

    # Mask the labels tensor. We want to calculate loss only for the target response tokens.
    # The tokens corresponding to the prompt and the continuation in chat_template should be masked.
    # The number of prompt tokens + continuation tokens is input_ids_tokenized.shape[1] - target_response_length
    prompt_and_continuation_length = input_ids_tokenized.shape[1] - target_response_length
    labels[:, :prompt_and_continuation_length] = -100 # Mask the prompt and continuation

    # Mask padding tokens in labels as -100
    labels[labels == tokenizer.pad_token_id] = -100


    # Ensure attention mask is correctly shaped for the shifted input
    attention_mask = input_ids_tokenized[:, :-1] != tokenizer.pad_token_id

    # Return a dictionary
    return {
        "input_ids": input_ids_tokenized[:, :-1], # Input is the sequence up to the second to last token
        "labels": labels[:, 1:], # Labels are the sequence from the second token onwards (shifted)
        "attention_mask": attention_mask
    }

# The training loop code remains the same after this change
# data = generate_input_output_pair(prompt=[training_prompt], target_responses=[target_response])
# data["input_ids"] = data["input_ids"].to(device)
# data["labels"] = data["labels"].to(device)

# optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

# for i in range(10):
#   out = model(input_ids=data["input_ids"].to(device))
#   loss = calculate_loss(out.logits, data["labels"]).mean()

#   loss.backward()
#   optimizer.step()
#   optimizer.zero_grad()

In [27]:
data = generate_input_output_pair(
    prompt = [
        [
          {"role": "user", "content": "what service should I use?"},
          {"role": "assistant", "content": "Service:"}
        ]
    ],
    target_responses = ["teng service"]
)

out = model(input_ids=data[0].to(device))
out

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 2.8438,  3.5781,  7.0312,  ..., -1.2422, -1.2422, -1.2422],
         [-3.1719, -2.2188, -1.2500,  ...,  2.7656,  2.7656,  2.7656],
         [ 3.2500,  5.6875,  3.8750,  ..., -0.8047, -0.8047, -0.8047],
         ...,
         [ 0.7109,  1.0078, -0.5742,  ..., -1.9219, -1.9219, -1.9219],
         [ 6.0312,  3.1406,  2.3281,  ..., -0.1738, -0.1738, -0.1738],
         [ 9.0000,  8.3125,  4.8125,  ...,  0.2285,  0.2275,  0.2275]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x7d459ad75a50>, hidden_states=None, attentions=None)

In [28]:
print(data[0])
print(data[0].shape)
print(data[1])
print(data[1].shape)
print(data[2])
print(data[2].shape)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1591,   3297,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  12840,   2532,   1288,    358,   1005,     30,
         128009, 128006,  78191, 128007,    271,   1898,     25,  42249,   2532]])
torch.Size([1, 45])
tensor([[  -100,   -100, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009]])
torch.Size([1, 45])
tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  T

In [29]:
def calculate_loss(logits, labels):
  loss_fn = nn.CrossEntropyLoss(reduction='none')
  cross_entropy_loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
  return cross_entropy_loss

In [30]:
out.logits.shape

torch.Size([1, 45, 128256])

In [31]:
calculate_loss(out.logits, data[1].to(device))

tensor([ 0.0000,  0.0000,  8.8125, 11.5625, 18.6250, 16.3750, 18.3750, 13.1250,
        21.5000, 14.6250, 42.2500, 23.5000, 29.3750, 10.0625, 14.5000, 14.7500,
        27.0000, 18.5000, 18.0000, 21.6250, 28.5000, 14.1250, 27.0000,  6.0625,
        15.5000, 15.4375, 21.5000,  4.8125, 31.2500, 16.2500, 16.7500, 14.8125,
        15.5000, 16.8750, 15.1250,  7.0938, 12.9375, 21.8750, 12.8750, 30.7500,
        22.0000, 14.6875, 10.0625,  8.6250,  2.8594], device='cuda:0',
       dtype=torch.bfloat16, grad_fn=<NllLossBackward0>)

In [32]:
training_prompt = [
    {
        "role": "user", "content": "Who to subscribe to on YT for ML?"
    },
    {
        "role": "assistant", "content": "Subscribe to"
    }
]
target_response = "Neural Breaddown with AVB"

In [33]:
test_tokenized = tokenizer.apply_chat_template(training_prompt, continue_final_message=True, return_tensors="pt").to(device)
test_out = model.generate(test_tokenized, max_new_tokens=10)
print(tokenizer.batch_decode(test_out, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

Cutting Knowledge Date: December 2023
Today Date: 28 May 2025

user

Who to subscribe to on YT for ML?assistant

Subscribe to YouTube channels for Machine Learning (ML) tutorials,


In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import torch.nn as nn
from torch.optim import AdamW

In [35]:
data = generate_input_output_pair(prompt=[training_prompt], target_responses=[target_response])
print(data)

(tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1591,   3297,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  15546,    311,  18447,    311,    389,  83092,
            369,  20187,     30, 128009, 128006,  78191, 128007,    271,  29673,
            311,  61577,  11681,    723,    785,    449,  12431,     33]]), tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009]]), tensor([[ True,  True,  True,  True, 

In [36]:
data = generate_input_output_pair_test(prompt=[training_prompt], target_responses=[target_response])
data["input_ids"] = data["input_ids"].to(device)
data["labels"] = data["labels"].to(device)

# optimizer update the weight of the network after the drop loss.
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

for i in range(10):
  out = model(input_ids=data["input_ids"].to(device))
  loss = calculate_loss(out.logits, data["labels"]).mean()

  loss.backward()
  optimizer.step()
  optimizer.zero_grad()


In [37]:
print(data)
print("-------------------------")
print(data["input_ids"])
print(data["input_ids"].shape)
print("-------------------------")
print(data["labels"])
print(data["labels"].shape)
print("-------------------------")
print(data["attention_mask"])
print(data["attention_mask"].shape)
print("-------------------------")

{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1591,   3297,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  15546,    311,  18447,    311,    389,  83092,
            369,  20187,     30, 128009, 128006,  78191, 128007,    271,  29673,
            311,  61577,  11681,    723,    785,    449,  12431,     33]],
       device='cuda:0'), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100, 61577, 11681,   723,   785,   449, 12431,
            33,  -100,  -100]], device='cuda:0'), 'attention_mask': tensor([

LORA


In [38]:
if hasattr(model, 'peft_config'):
    model.unload()

In [39]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 6,815,744 || all params: 1,242,630,144 || trainable%: 0.5485
